In [1]:
# @title
# pip install yfinance plotly pandas numpy
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
#@title Seleccione Acción y Medias Móviles {run:'auto'}
ticker= "NVDA" #@param ["AAPL", "PLTR", "MSFT", "NVDA", "GOOGL", "AMZN", "META", "TSM","BRK.B", "V", "JPM", "XOM", "LLY", "MRK", "UNH", "PG", "MA","CVX", "KO", "PEP", "COST", "TMO", "ORCL", "CSCO", "NKE", "VZ", "ASML", "TXN", "ABT", "TM", "SAP", "AMD", "NFLX", "NOW", "ADBE", "LVMUY", "BABA", "SHEL", "TMUS", "QCOM", "PFE", "SNY", "AZN", "TOT", "GSK", "RIO", "BHP", "MCD​"]
#ticker_symbol = "BTC-USD" #@param ["BTC-USD", "ETH-USD", "USDT-USD", "XRP-USD", "LTC-USD", "ADA-USD", "DOT-USD", "BCH-USD", "XLM-USD", "LINK-USD"]
YEARS = 1 #@param {type:"integer"}
end_date   =  datetime.today().strftime("%Y-%m-%d")
start_date = (datetime.today() - relativedelta(years=YEARS)).strftime("%Y-%m-%d")
MA_RAPIDA = 10 #@param {type:"integer"}
MA_LENTA  = 20 #@param {type:"integer"}
MA_CORR   = 100 #@param {type:"integer"}

In [3]:
# @title
# Parámetros
#ticker = "NVDA"
#YEARS = 1
#end_date = datetime.today().strftime("%Y-%m-%d")
#start_date = (datetime.today() - relativedelta(years=YEARS)).strftime("%Y-%m-%d")


#MA_RAPIDA = 10   # EMA rápida (dorada) default 10
#MA_LENTA  = 20   # EMA lenta (azul) default 20
#MA_CORR   = 100   # EMA correlación (roja) default 100
SLOPE_WIN = 9  # ventana para calcular pendiente default 9
MIN_SLOPE_PCT = 1.0 # 1.0  pendiente mínima del 1% (equivalente a >45° en términos prácticos) 
                    # porque uso una formula de rendimiento donde rendimiento del 100% 
                    # significa que aumenta 1 unidad en 1 periodo de tiempo => pendiente de 45°

# --- Descarga de datos ---
df = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True, progress=False)
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)

# --- EMAs ---
df["EMA_fast"] = df["Close"].ewm(span=MA_RAPIDA, adjust=False).mean()
df["EMA_slow"] = df["Close"].ewm(span=MA_LENTA,  adjust=False).mean()
df["EMA_corr"] = df["Close"].ewm(span=MA_CORR,  adjust=False).mean()


# --- Golden cross: EMA rápida cruza arriba de EMA lenta ---
cross_up = (df["EMA_fast"] > df["EMA_slow"]) & (df["EMA_fast"].shift(1) <= df["EMA_slow"].shift(1))

# --- Cálculo de pendiente de EMA rápida (en porcentaje) ---
# Pendiente porcentual promedio de los últimos SLOPE_WIN días
df["slope_pct"] = ((df["EMA_fast"] - df["EMA_fast"].shift(SLOPE_WIN)) /
                   df["EMA_fast"].shift(SLOPE_WIN)) * 100

# Filtramos cruces con pendiente positiva y significativa
steep_slope = df["slope_pct"] > MIN_SLOPE_PCT
valid_cross = cross_up & steep_slope

# --- Gráfico Plotly ---
fig = go.Figure()

# Velas
fig.add_trace(go.Candlestick(
    x=df.index, open=df["Open"], high=df["High"], low=df["Low"], close=df["Close"],
    name="OHLC",
    increasing_line_color="#26a69a", decreasing_line_color="#ef5350",
    opacity=0.6
))

# EMA rápida (dorada)
fig.add_trace(go.Scatter(
    x=df.index, y=df["EMA_fast"],
    mode="lines", name=f"EMA {MA_RAPIDA}",
    line=dict(color="goldenrod", width=2),
    hovertemplate="EMA rápida: %{y:.2f}<extra></extra>"
))

# EMA lenta (azul)
fig.add_trace(go.Scatter(
    x=df.index, y=df["EMA_slow"],
    mode="lines", name=f"EMA {MA_LENTA}",
    line=dict(color="blue", width=2),
    hovertemplate="EMA lenta: %{y:.2f}<extra></extra>"
))

# EMA lenta (purple)
fig.add_trace(go.Scatter(
    x=df.index, y=df["EMA_corr"],
    mode="lines", name=f"EMA {MA_CORR}",
    line=dict(color="purple", width=2),
    hovertemplate="EMA Corr: %{y:.2f}<extra></extra>"
))

# Marcadores en golden cross válidos
cross_idx = df.index[valid_cross.fillna(False)]
if len(cross_idx) > 0:
    fig.add_trace(go.Scatter(
        x=cross_idx,
        y=df.loc[cross_idx, "EMA_fast"],
        mode="markers", name=f"Golden cross (> {MIN_SLOPE_PCT}% pendiente)",
        marker=dict(symbol="star", size=12, line=dict(width=2, color="black"), color="orange"),
        customdata=np.stack([df.loc[cross_idx, "slope_pct"]], axis=1),
        hovertemplate="%{x|%Y-%m-%d}<br>EMA rápida: %{y:.2f}<br>Pendiente: %{customdata[0]:.2f}%<extra></extra>"
    ))

# Layout
fig.update_layout(
    title=f"{ticker} — Golden Cross con pendiente > {MIN_SLOPE_PCT}% · {start_date} → {end_date}",
    xaxis_title="Fecha", yaxis_title="Precio (USD)",
    hovermode="x unified",
    xaxis=dict(rangeslider=dict(visible=True), type="date"),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
    width=1200, height=700
)

# Mostrar información adicional
print(f"Golden crosses detectados con pendiente > {MIN_SLOPE_PCT}%: {len(cross_idx)}")
for date in cross_idx:
    slope = df.loc[date, "slope_pct"]
    ema_fast = df.loc[date, "EMA_fast"]
    ema_slow = df.loc[date, "EMA_slow"]
    print(f"  {date.strftime('%Y-%m-%d')}: EMA rápida={ema_fast:.2f}, EMA lenta={ema_slow:.2f}, Pendiente={slope:.2f}%")

fig.show()

# Si no hay cruces válidos, mostramos todos los golden crosses para diagnóstico
if len(cross_idx) == 0:
    all_cross_idx = df.index[cross_up.fillna(False)]
    print(f"\nTodos los golden crosses encontrados (sin filtro de pendiente): {len(all_cross_idx)}")
    for date in all_cross_idx:
        slope = df.loc[date, "slope_pct"]
        ema_fast = df.loc[date, "EMA_fast"]
        ema_slow = df.loc[date, "EMA_slow"]
        print(f"  {date.strftime('%Y-%m-%d')}: EMA rápida={ema_fast:.2f}, EMA lenta={ema_slow:.2f}, Pendiente={slope:.2f}%")

    # Sugerencia para ajustar el parámetro
    if len(all_cross_idx) > 0:
        avg_slope = df.loc[all_cross_idx, "slope_pct"].mean()
        print(f"\nSugerencia: Intenta con MIN_SLOPE_PCT = {max(0.1, avg_slope * 0.5):.2f}")

Golden crosses detectados con pendiente > 1.0%: 3
  2025-01-03: EMA rápida=138.16, EMA lenta=137.81, Pendiente=3.10%
  2025-02-14: EMA rápida=131.90, EMA lenta=131.61, Pendiente=3.67%
  2025-05-01: EMA rápida=107.80, EMA lenta=107.80, Pendiente=1.05%
